In [1]:
'''Data Dictionary - 

business_code - company code of the account
cust_number - customer number given to all the customers of the Account
name_customer - name of the customer
clear_date - The date on which the customer clears an invoice, or in simple terms, they make the full payment.
buisness_year - 
doc_id - It is also an unique identifier of an invoice is a primary key for acct_doc_header table
posting_date - 
document_create_date - The date on which the invoice document was created
document_create_date.1 - 
due_in_date - The date on which the customer is expected to clear an invoice
invoice_currency - The currency of the invoice amount in the document for the invoice
document type - It represents the type of document. eg D1 represents Invoice
posting_id - key indicator to identify whether an AR item is invoice, deduction, credit memo based on its value. Apllicable for SAP ERP
area_business - Business area in sap is defined as an organisationalarea within the financial accounting module.
total_open_amount - The amount that is yet to be paid for that invoice
baseline_create_date - The date on which the Invoice was created.
cust_payment_terms - Business terms and agreements between customers and accounts on discounts and days of payment
invoice_id - Unique number assigned when a seller creates an Invoice.
isOpen - indicator of whether an invoice is open or closed. isopen = 1, means the invoice is open.
'''

'Data Dictionary - \n\nbusiness_code - company code of the account\ncust_number - customer number given to all the customers of the Account\nname_customer - name of the customer\nclear_date - The date on which the customer clears an invoice, or in simple terms, they make the full payment.\nbuisness_year - \ndoc_id - It is also an unique identifier of an invoice is a primary key for acct_doc_header table\nposting_date - \ndocument_create_date - The date on which the invoice document was created\ndocument_create_date.1 - \ndue_in_date - The date on which the customer is expected to clear an invoice\ninvoice_currency - The currency of the invoice amount in the document for the invoice\ndocument type - It represents the type of document. eg D1 represents Invoice\nposting_id - key indicator to identify whether an AR item is invoice, deduction, credit memo based on its value. Apllicable for SAP ERP\narea_business - Business area in sap is defined as an organisationalarea within the financial

In [5]:
#Importing Libraries
import pandas as pd
import numpy as np
import datetime

In [6]:
data = pd.read_csv('1828166.csv') #Reads the data file 

In [7]:
data.buisness_year = data.buisness_year.astype(int)

In [8]:
for i in data.columns:
    if "date" in i:
        print(f"'{i}',")

'clear_date',
'posting_date',
'document_create_date',
'document_create_date.1',
'due_in_date',
'baseline_create_date',


In [9]:
data.clear_date = pd.to_datetime(data.clear_date,errors="coerce")
data.posting_date = pd.to_datetime(data.posting_date,errors="coerce")
data['document_create_date'] = pd.to_datetime(data['document_create_date'], format='%Y%m%d')
data['document_create_date.1'] = pd.to_datetime(data['document_create_date.1'], format='%Y%m%d')
data['due_in_date'] = pd.to_datetime(data['due_in_date'], format='%Y%m%d')
data['baseline_create_date'] = pd.to_datetime(data['baseline_create_date'], format='%Y%m%d')

In [10]:
data["document type"].value_counts()

RV    49994
X2        6
Name: document type, dtype: int64

In [11]:
data = data[data["document type"]=="RV"]

In [12]:
test_data = data[data["isOpen"]==1]
train_data = data[data["isOpen"]==0]
print(test_data.shape)
print(train_data.shape)

(3447, 19)
(46547, 19)


In [13]:
train_data["days_to_pay"] = (train_data["clear_date"] - train_data["posting_date"]).dt.days

<ipython-input-13-842f18acb0f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["days_to_pay"] = (train_data["clear_date"] - train_data["posting_date"]).dt.days


In [14]:
mapper = train_data.groupby(by="cust_number")["days_to_pay"].mean().reset_index()

In [15]:
overall_average = train_data.days_to_pay.mean()
overall_average

18.165209358283025

In [16]:
test_cust = test_data.cust_number.unique()
train_cust = train_data.cust_number.unique()

In [17]:
test_data[test_data["posting_date"].isnull()]

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,invoice_currency,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen


In [18]:
# for i in test_cust:
#     if i not in train_cust:
#         print(i)

In [19]:
result = pd.merge(test_data, mapper, on="cust_number", how = "left")

In [20]:
result["days_to_pay"] = np.where(result["days_to_pay"].isnull(), overall_average, result["days_to_pay"])

In [21]:
result["prediction_date"] = result["posting_date"] + pd.to_timedelta(round(result["days_to_pay"]), unit='D')
result

,business_code,cust_number,name_customer,clear_date,buisness_year,doc_id,posting_date,document_create_date,document_create_date.1,due_in_date,...,document type,posting_id,area_business,total_open_amount,baseline_create_date,cust_payment_terms,invoice_id,isOpen,days_to_pay,prediction_date
0,U001,0200769623,WAL-MAR us,NaT,2020,1930778940.0,2020-04-12,2020-04-11,2020-04-12,2020-04-27,...,RV,1.0,NaN,13815.36,2020-04-12,NAH4,1.930779e+09,1,12.794509,2020-04-25
1,U001,0200936373,SUPE corporation,NaT,2020,1930739469.0,2020-04-02,2020-04-03,2020-04-02,2020-04-17,...,RV,1.0,NaN,88346.44,2020-04-02,NAA8,1.930739e+09,1,18.962644,2020-04-21
2,U001,0200769623,WAL-MAR trust,NaT,2020,1930675227.0,2020-03-20,2020-03-20,2020-03-20,2020-04-04,...,RV,1.0,NaN,12965.69,2020-03-20,NAH4,1.930675e+09,1,12.794509,2020-04-02
3,U001,0200769623,WAL-MAR systems,NaT,2020,1930725711.0,2020-04-01,2020-03-31,2020-04-01,2020-04-16,...,RV,1.0,NaN,14323.58,2020-04-01,NAH4,1.930726e+09,1,12.794509,2020-04-14
4,U013,CC6530,HEINZ corporation,NaT,2020,1991841230.0,2020-03-17,2020-03-17,2020-03-17,2020-05-01,...,RV,1.0,NaN,16216.22,2020-03-17,NAVF,1.991841e+09,1,67.578947,2020-05-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,U001,0200407025,ALBERT in,NaT,2020,1930778416.0,2020-04-14,2020-04-11,2020-04-14,2020-04-29,...,RV,1.0,NaN,122734.80,2020-04-14,NAA8,1.930778e+09,1,22.131579,2020-05-06
3443,U001,0200769623,WAL-MAR associates,NaT,2020,1930772421.0,2020-04-10,2020-04-09,2020-04-10,2020-04-25,...,RV,1.0,NaN,50045.40,2020-04-10,NAH4,1.930772e+09,1,12.794509,2020-04-23
3444,U001,0200769623,WAL-MAR,NaT,2020,1930659564.0,2020-03-17,2020-03-16,2020-03-17,2020-04-01,...,RV,1.0,NaN,16067.93,2020-03-17,NAH4,1.930660e+09,1,12.794509,2020-03-30
3445,U001,0200769623,WAL-MAR in,NaT,2020,1930827753.0,2020-04-27,2020-04-25,2020-04-27,2020-05-12,...,RV,1.0,NaN,36834.06,2020-04-27,NAH4,1.930828e+09,1,12.794509,2020-05-10
